In [70]:
#pip install --upgrade prophet
#!pip install cloudpickle


In [183]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

import pickle

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier

import numpy as np


In [139]:
df = pd.read_csv('epl_season_1993_2024.csv')
df.tail()

,Season_End_Year,Total_Matches,Total_HomeGoals,Total_AwayGoals,Home_Wins,Away_Wins,Draws,Total_Goals,Champion,Runners
27,2020,380,576,458,172,116,92,1034,Liverpool,Manchester City
28,2021,380,514,510,144,153,83,1024,Manchester City,Manchester United
29,2022,380,575,496,163,129,88,1071,Manchester City,Liverpool
30,2023,380,621,463,184,109,87,1084,Manchester City,Arsenal
31,2024,380,684,562,175,123,82,1246,Manchester City,Arsenal


In [182]:
# Focus on Home Goals and Away Goals
home_goals = df['Total_HomeGoals']
away_goals = df['Total_AwayGoals']

# Split the data into training and testing sets
train_home, test_home = train_test_split(home_goals, test_size=0.2, shuffle=False)
train_away, test_away = train_test_split(away_goals, test_size=0.2, shuffle=False)

# Function to perform grid search for ARIMA
def grid_search_arima(train_data):
    best_aic = np.inf
    best_order = None
    best_model = None

    for p in range(0, 4):
        for d in range(0, 3):
            for q in range(0, 4):
                try:
                    model = ARIMA(train_data, order=(p, d, q)).fit()
                    aic = model.aic
                    if aic < best_aic:
                        best_aic = aic
                        best_order = (p, d, q)
                        best_model = model
                except:
                    continue
    return best_order, best_model

# Perform grid search for Home Goals
best_order_home, best_model_home = grid_search_arima(train_home)
print(f'Best ARIMA order for Home Goals: {best_order_home}')

# Save the Home Goals model
with open('model pickel/best_model_home_goals.pkl', 'wb') as f:
    pickle.dump(best_model_home, f)


# Perform grid search for Away Goals
best_order_away, best_model_away = grid_search_arima(train_away)
print(f'Best ARIMA order for Away Goals: {best_order_away}')

# Save the Away Goals model
with open('model pickel/best_model_away_goals.pkl', 'wb') as f:
    pickle.dump(best_model_away, f)


/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting param

Best ARIMA order for Home Goals: (1, 2, 1)


/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("M

Best ARIMA order for Away Goals: (0, 2, 1)


/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/pranavsrinivasvenkatesh/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [185]:
# Calculate historical win and runner-up rates
champion_counts = df['Champion'].value_counts()
runner_up_counts = df['Runners'].value_counts()

# Calculate the probabilities
total_seasons = len(df)
champion_prob = champion_counts / total_seasons
runner_up_prob = runner_up_counts / total_seasons

# Save the probabilities
with open('model pickel/champion_probabilities.pkl', 'wb') as f:
    pickle.dump(champion_prob, f)
with open('model pickel/runner_up_probabilities.pkl', 'wb') as f:
    pickle.dump(runner_up_prob, f)

In [180]:

# Forecast using the best models
future_periods = 10 # Example: forecast 5 periods ahead
forecast_home = best_model_home.forecast(steps=future_periods)
forecast_away = best_model_away.forecast(steps=future_periods)

# Compile the forecasted data into a DataFrame
last_year = df.index.max()
future_years = np.arange(int(last_year) + 1, int(last_year) + 1 + future_periods)

predictions = pd.DataFrame({
    "Season_End_Year": future_years.astype(int),
    "Predicted_HomeGoals": np.ceil(forecast_home).astype(int),
    "Predicted_AwayGoals": np.ceil(forecast_away).astype(int)
})

# Calculate the total goals as the sum of home and away goals
predictions['Predicted_Total_Goals'] = predictions['Predicted_HomeGoals'] + predictions['Predicted_AwayGoals']



In [181]:
predictions

,Season_End_Year,Predicted_HomeGoals,Predicted_AwayGoals,Predicted_Total_Goals
25,32,589,456,1045
26,33,596,454,1050
27,34,592,452,1044
28,35,593,451,1044
29,36,591,449,1040
30,37,591,447,1038
31,38,590,445,1035
32,39,589,444,1033
33,40,589,442,1031
34,41,588,440,1028
